In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop,Adam
import tensorflow_addons as tfa
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from tensorflow.keras import layers,regularizers,Sequential,Model,backend,callbacks,optimizers,metrics,losses
import tensorflow as tf
import sys
sys.path.append('../input/iterativestratification/iterative_stratification-0.1.6-py3-none-any.whl')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

df = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
non_ctl_idx = df.loc[df['cp_type']!='ctl_vehicle'].index.to_list()
df = df.drop(['sig_id','cp_type','cp_dose','cp_time'],axis=1)
data_X_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
data_X_test = data_X_test.drop(['sig_id','cp_dose','cp_time'],axis=1)

dada = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
dada = dada.drop('sig_id',axis=1)
df_label = dada.values
df = df.iloc[non_ctl_idx]
df_label = df_label[non_ctl_idx]

In [ ]:


cs = df.columns.str.contains('c-')
gs = df.columns.str.contains('g-')

def preprocessor(train,test):
    
    
    n_gs = 2 
    n_cs = 100 
    
    pca_cs = PCA(n_components = n_cs)
    pca_gs = PCA(n_components = n_gs)

    train_pca_gs = pca_gs.fit_transform(train[:,gs])
    train_pca_cs = pca_cs.fit_transform(train[:,cs])
    test_pca_gs = pca_gs.transform(test[:,gs])
    test_pca_cs = pca_cs.transform(test[:,cs])

    
    train_c_mean = train[:,cs].mean(axis=1)
    test_c_mean = test[:,cs].mean(axis=1)
    train_g_mean = train[:,gs].mean(axis=1)
    test_g_mean = test[:,gs].mean(axis=1)
    
    
    train = np.concatenate((train,train_pca_gs,train_pca_cs,train_c_mean[:,np.newaxis]
                            ,train_g_mean[:,np.newaxis]),axis=1)
    test = np.concatenate((test,test_pca_gs,test_pca_cs,test_c_mean[:,np.newaxis],
                           test_g_mean[:,np.newaxis]),axis=1)
    
    scaler = preprocessing.StandardScaler()

    train = scaler.fit_transform(train)

    test = scaler.transform(test)
    
    return train, test

In [ ]:

p_min = 0.0005
p_max = 0.9995

def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -backend.mean(y_true*backend.log(y_pred) + (1-y_true)*backend.log(1-y_pred))

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_logloss', factor=0.1, verbose=0,mode='min',
                              patience=3, min_lr=1E-7)
early_st = tf.keras.callbacks.EarlyStopping(monitor='val_logloss', min_delta=1E-5, patience=7, verbose=0, mode='min',
    baseline=None, restore_best_weights=True)

In [ ]:
def create_model1(num_columns):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(num_columns),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(512, activation="elu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="elu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(512, activation="elu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="elu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),   
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid"))
    ])
    model.compile(optimizer=tfa.optimizers.Lookahead(tf.optimizers.Adam(), sync_period=10),
                  loss=losses.BinaryCrossentropy(label_smoothing=0.0015),metrics=logloss
                  )
    return model

In [ ]:
def create_model(num_columns):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(num_columns),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5), 
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(512, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),        
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid"))
    ])
    model.compile(optimizer=tfa.optimizers.Lookahead(tf.optimizers.Adam(), sync_period=10),
                  loss=losses.BinaryCrossentropy(label_smoothing=0.001),metrics=logloss
                  )
    return model

In [ ]:
X = 976

from keras import backend 
n_seeds = 6
np.random.seed(1)
seeds = np.random.randint(0,100,size=n_seeds)

n_labels = df_label.shape[1]
n_train = df.shape[0]
n_test = data_X_test.shape[0]
n_folds = 5
y_pred = np.zeros((n_test,n_labels))
oof = tf.constant(0.0)
hists = []


for seed in seeds:
    fold = 0
    print(fold)
    mlkf = MultilabelStratifiedKFold(n_splits=n_folds,shuffle=True,random_state=seed)
    for n,(train, test) in enumerate(mlkf.split(df,df_label)):
        tf.print('OOF score is ',oof)
        print(f'Fold {n}')
        
        
        train_x, validation_x = preprocessor(df.iloc[train].values,
                                       df.iloc[test].values)
        _,data_test = preprocessor(df.iloc[train].values,
                                   data_X_test.drop('cp_type',axis=1).values)

        train_y = df_label[train]
        validation_y = df_label[test]
        
        if seed%2 == 0:
            model = create_model1(X)
        else:    
            model = create_model(X)

        model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.001), metrics=logloss )
        

        hist = model.fit(train_x,train_y, batch_size=128, epochs=200,verbose=0,validation_data = (validation_x,validation_y),callbacks =[reduce_lr, early_st])
        hists.append(hist)
        
        # Save Model
        model.save('_seed_'+str(seed)+'_fold_'+str(fold))

        # OOF Score
        y_val = model.predict(validation_x)
        oof += logloss(tf.constant(validation_y,dtype=tf.float32),tf.constant(y_val,dtype=tf.float32))/(n_folds*n_seeds)

        # Run prediction
        y_pred += model.predict(data_test)/(n_folds*n_seeds)
        
        fold += 1
        


In [ ]:
tf.print('OOF score is ',oof)

plt.figure(figsize=(12,8))

hist_trains = []
hist_lens = []
for i in range(n_folds*n_seeds):
    hist_train = (hists[i]).history['logloss']
    hist_trains.append(hist_train)
    hist_lens.append(len(hist_train))
hist_train = []
for i in range(min(hist_lens)):
    hist_train.append(np.mean([hist_trains[j][i] for j in range(n_folds*n_seeds)]))

plt.plot(hist_train)

hist_vals = []
hist_lens = []
for i in range(n_folds*n_seeds):
    hist_val = (hists[i]).history['val_logloss']
    hist_vals.append(hist_val)
    hist_lens.append(len(hist_val))
hist_val = []
for i in range(min(hist_lens)):
    hist_val.append(np.mean([hist_vals[j][i] for j in range(n_folds*n_seeds)]))

plt.plot(hist_val)

plt.yscale('log')
plt.yticks(ticks=[1,1E-1,1E-2])
plt.xlabel('Epochs')
plt.ylabel('Average Logloss')
plt.legend(['Training','Validation'])

In [ ]:

final = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
final.iloc[:,1:] = np.clip(y_pred,p_min,p_max)
final.iloc[data_X_test['cp_type'] == 'ctl_vehicle',1:] = 0

final.to_csv('submission.csv', index=False)
final.head()